This notebook creates a single visualization showing all of the results from the BWM paper in one "view" so to speak

It first generates just the left sided areas. The user explodes them, and then it captures a screenshot.

Then using the same point of view, it plots all of the major results

In [1]:
import pandas as pd
import oursin as urchin
import matplotlib.pyplot as plt
import numpy as np

In [2]:
urchin.setup()

(URN) connected to server
Login sent with ID: Dan


In [4]:
ds_stim = pd.read_parquet('./stimulus_bwm.pqt')
ds_choice = pd.read_parquet('./choice_bwm.pqt')
ds_feedback = pd.read_parquet('./feedback_bwm.pqt')
ds_wheel_spd = pd.read_parquet('./wheel_speed_bwm.pqt')
ds_block = pd.read_parquet('./block_bwm.pqt')

In [5]:
ds_stim

,acronym_b,euclidean_effect,euclidean_latency,euclidean_significant,glm_effect,mannwhitney_effect,mannwhitney_significant,decoding_effect,decoding_frac_significant,decoding_significant,atlas_id_b
0,LP,0.204980,0.044366,True,0.000445,0.012835,True,0.035892,0.363636,True,-218
1,MG,0.172636,0.050704,False,0.000199,0.000000,False,0.010562,0.142857,False,-475
2,PoT,0.182038,NaN,False,0.000241,0.000000,False,0.012971,0.000000,False,-563807435
3,SGN,0.415604,0.050704,True,0.000225,0.000000,False,0.008863,0.400000,False,-325
4,ACB,0.119662,0.109859,True,0.000557,0.000000,False,0.002423,0.000000,False,-56
...,...,...,...,...,...,...,...,...,...,...,...
118,SMT,0.206797,0.069718,False,0.000238,0.000000,False,0.031524,0.500000,False,-366
119,AM,0.129519,0.111972,False,0.000120,0.000000,False,0.034519,0.000000,False,-127
120,CEA,0.089291,0.103521,False,0.000055,0.000000,False,0.026964,0.000000,False,-536
121,EPv,0.146871,0.014789,False,0.000860,0.000000,False,-0.003401,0.000000,False,-966


In [7]:
# before doing anything, load all the areas in their default colors and render
# this way we can make a picture w/ just the regions (and put labels on them)
area_vis = {}

for i, row in ds_stim.iterrows():
    aname = row.acronym_b + '-lh'
    area_vis[aname] = True

urchin.ccf.set_visibility(area_vis)

In [9]:
urchin.camera.set_rotation(35,0,135)

In [13]:
urchin.camera.capture_image('./imgs/default_exploded.png')

(Camera receive meta) 2
(Camera) received 1000000 bytes
(Camera) received 343428 bytes
(Camera received all data)


In [23]:
def rgb2hex(rgba):
    r, g, b, a = rgba
    r = int(r * 255)
    g = int(g * 255)
    b = int(b * 255)
    hex_code = "#{:02x}{:02x}{:02x}".format(r, g, b)
    return hex_code

def stim_color(v, cmap, min_val, max_val):
    return rgb2hex(cmap((v - min_val) / (max_val - min_val)))

In [24]:
# get the colormap (greens)
sig_idxs = np.argwhere(ds_stim.decoding_significant.values == True)
min_val = np.min(ds_stim.decoding_effect.values[sig_idxs])
max_val = np.max(ds_stim.decoding_effect.values[sig_idxs])
cmap = plt.get_cmap('YlGn')

area_vis = {}
area_color = {}

for i, row in ds_stim.iterrows():
    aname = row.acronym_b + '-lh'
    area_vis[aname] = True
    if row.decoding_significant:
        area_color[aname] = stim_color(row.decoding_effect, cmap, min_val, max_val)
    else:
        area_color[aname] = '#ffffff'

urchin.ccf.set_visibility(area_vis)
urchin.ccf.set_color(area_color)

urchin.camera.capture_image('./imgs/stimulus.png')

(Camera receive meta) 2
(Camera) received 1000000 bytes
(Camera) received 98011 bytes
(Camera received all data)


In [25]:
# get the colormap (greens)
sig_idxs = np.argwhere(ds_choice.decoding_significant.values == True)
min_val = np.min(ds_choice.decoding_effect.values[sig_idxs])
max_val = np.max(ds_choice.decoding_effect.values[sig_idxs])
cmap = plt.get_cmap('YlOrRd')

area_vis = {}
area_color = {}

for i, row in ds_choice.iterrows():
    aname = row.acronym_b + '-lh'
    area_vis[aname] = True
    if row.decoding_significant:
        area_color[aname] = stim_color(row.decoding_effect, cmap, min_val, max_val)
    else:
        area_color[aname] = '#ffffff'

urchin.ccf.set_visibility(area_vis)
urchin.ccf.set_color(area_color)

urchin.camera.capture_image('./imgs/choice.png')

(Camera receive meta) 2
(Camera) received 1000000 bytes
(Camera) received 260091 bytes
(Camera received all data)


In [18]:
# get the colormap (greens)
sig_idxs = np.argwhere(ds_feedback.decoding_significant.values == True)
min_val = np.min(ds_feedback.decoding_effect.values[sig_idxs])
max_val = np.max(ds_feedback.decoding_effect.values[sig_idxs])
cmap = plt.get_cmap('Reds')

area_vis = {}
area_color = {}

for i, row in ds_feedback.iterrows():
    aname = row.acronym_b + '-lh'
    area_vis[aname] = True
    if row.decoding_significant:
        area_color[aname] = stim_color(row.decoding_effect, cmap, min_val, max_val)
    else:
        area_color[aname] = '#ffffff'

urchin.ccf.set_visibility(area_vis)
urchin.ccf.set_color(area_color)

urchin.camera.capture_image('./imgs/feedback.png')

(Camera receive meta) 2
(Camera) received 1000000 bytes
(Camera) received 285366 bytes
(Camera received all data)


In [28]:
# get the colormap (greens)
sig_idxs = np.argwhere(ds_block.decoding_significant.values == True)
min_val = 0
max_val = np.max(ds_block.decoding_effect.values[sig_idxs])
cmap = plt.get_cmap('Purples')

area_vis = {}
area_color = {}

for i, row in ds_block.iterrows():
    aname = row.acronym_b + '-lh'
    area_vis[aname] = True
    if row.decoding_significant:
        area_color[aname] = stim_color(row.decoding_effect, cmap, min_val, max_val)
    else:
        area_color[aname] = '#ffffff'

urchin.ccf.set_visibility(area_vis)
urchin.ccf.set_color(area_color)

urchin.camera.capture_image('./imgs/block.png')

(Camera receive meta) 2
(Camera) received 1000000 bytes
(Camera) received 47408 bytes
(Camera received all data)


In [29]:
# get the colormap (greens)
sig_idxs = np.argwhere(ds_wheel_spd.decoding_significant.values == True)
min_val = np.min(ds_wheel_spd.decoding_effect.values[sig_idxs])
max_val = np.max(ds_wheel_spd.decoding_effect.values[sig_idxs])
cmap = plt.get_cmap('Blues')

area_vis = {}
area_color = {}

for i, row in ds_wheel_spd.iterrows():
    aname = row.acronym_b + '-lh'
    area_vis[aname] = True
    if row.decoding_significant:
        area_color[aname] = stim_color(row.decoding_effect, cmap, min_val, max_val)
    else:
        area_color[aname] = '#ffffff'

urchin.ccf.set_visibility(area_vis)
urchin.ccf.set_color(area_color)

urchin.camera.capture_image('./imgs/wheel_spd.png')

(Camera receive meta) 2
(Camera) received 1000000 bytes
(Camera) received 291191 bytes
(Camera received all data)
